#### **Linear Regression** aims to find the best-fit line that minimizes the error between predicted and actual values. Mathematically, this involves:
#### - Predicting $y$ as $$\hat{y} = w_1x + w_0$$ (for one feature).
#### - Minimizing the cost function $$J(w) = \frac{1}{2m} \sum_{i=1}^m (\hat{y}_i - y_i)^2$$, where $m$ is the number of data points.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
x =np.array([1, 2, 3, 4, 5]) # Features
y = np.array([1.5, 3.1, 4.5, 6.3, 7.8]) # Target
m = len(y)

In [3]:
def lin_reg(x, w, b):
    return w * x + b

In [4]:
def cost_function(x, y, w, b):
    predictions = lin_reg(x, w, b)
    cost = (1 / (2 * m)) * np.sum((predictions - y) ** 2)
    return cost

#### Compute gradients for $w$ and $b$:
#### $$\frac{\partial J}{\partial w} = \frac{1}{m} \sum (\hat{y} - y)x \qquad \text{and} \qquad \frac{\partial J}{\partial b} = \frac{1}{m} \sum (\hat{y} - y)$$
#### And update them iteratively:
#### $$w := w - \alpha \frac{\partial J}{\partial w} \qquad \text{and} \qquad b := b - \frac{\partial J}{\partial b}$$, where $\alpha$ is the learning rate

In [5]:
def gradient_descent(x, y, w, b, alpha, iter):
    for i in range(iter):
        predictions = lin_reg(x, w, b)
        error = predictions - y
        dw = (1 / m) * np.dot(error, x)
        db = (1 / m) * np.sum(error)

        # Update parameters
        w -= alpha * dw
        b -= alpha * db

        # Every 100 iterations print the cost
        if i % 100 == 0:
            cost = cost_function(x, y, w, b)
            print(f"Iteration {i}: Cost: {cost:.4f}")
    return w, b

In [6]:
# Initialization paramaters
w_init = 0 # Start with a slope of 0
b_init = 0 # Start wiht and intercept of 0
alpha = 0.01 # Learning rate
iterations = 1000

In [ ]:
# Train the model
w, b = gradient_descent(x, y, w_init, b_init, alpha, iterations)
print(f"Trained parameters: w = {w:.4f}, b = {b:.4f}")

In [ ]:
# Calculate final cost
final_cost = cost_function(x, y, w, b)
print(f"Final cost: {final_cost:.4f}")

In [ ]:
# Plot the data
plt.scatter(x, y, label="Data points", color="blue")
plt.plot(x, lin_reg(x, w, b), label="Regression line", color="red")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.show()

#### To extend **Linear Regression** to **multiple features** (also known as **Multivariate Linear Regression**), we follow a similar process but modify the mathematical representation and operations to handle vectorized data efficiently.
#### The hypothesis function becomes:
#### $$\hat{y} = X \cdot \mathbf{w} + b$$
#### Where:
#### - $ X $ is the $ m \times n $ matrix of input features ($ m $: samples, $ n $: features).
#### - $ \mathbf{w} $ is the $ n \times 1 $ vector of weights.
#### - $ b $ is the bias (scalar).

#### The cost function remains the Mean Squared Error:
#### $$J(\mathbf{w}, b) = \frac{1}{2m} \sum_{i=1}^m (\hat{y}_i - y_i)^2$$

In [10]:
X = np.array([[1, 2],
              [2, 3],
              [3, 4],
              [4, 5],
              [5, 6]]) # Features (5 samples with 2 features)
Y = np.array([5, 7, 9, 11, 13]) # Target
m, n = X.shape # m: number of samples, n: number of features

In [11]:
# Normalization (feature scaling)
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [12]:
def mult_lin_reg(X, w, b):
    return np.dot(X, w) + b

In [13]:
def mult_cost_function(X, Y, w, b):
    predictions = mult_lin_reg(X, w, b)
    cost = (1 / (2 * m)) * np.sum((predictions - Y) ** 2)
    return cost

In [14]:
def mult_grad_descent(X, Y, w, b, alpha, iter):
    for i in range(iter):
        predictions = mult_lin_reg(X, w, b)
        error = predictions - Y
        dw = (1 / m) * np.dot(X.T, error)
        db = (1 / m) * np.sum(error)

        # Update parameters
        w -= alpha * dw
        b -= alpha * db

        # Every 100 iterations print the cost
        if i % 100 == 0:
            cost = mult_cost_function(X, Y, w, b)
            print(f"Iteration {i}: Cost {cost:.4f}")
    return w, b

In [15]:
# Initialize parameters
w_init = np.zeros(n) # Weights [vector of zeros with shape (n, )]
b_init =0 # Bias (scalar)
alpha = 0.01
iterations = 1000

In [ ]:
# Train model
w, b = mult_grad_descent(X, Y, w_init, b_init, alpha, iterations)
print(f"Trained parameters: w = {w}, b = {b}")

In [ ]:
# Final cost
final_cost = mult_cost_function(X, Y, w, b)
print(f"Final cost: {final_cost:.4f}")

In [18]:
# Plot the data (with 2 features, a 3D plot can be used to show the fit)
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:, 0], X[:, 1], Y, color='blue', label='Data points')

# Generate a grid for the regression plane
x1, x2 = np.meshgrid(np.linspace(X[:, 0].min(), X[:, 0].max(), 10),
                     np.linspace(X[:, 1].min(), X[:, 1].max(), 10))

y_pred = w[0] * x1 + w[1] * x2 + b

ax.plot_surface(x1, x2, y_pred, color='red', alpha=0.5, label='Regression plane')

ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_zlabel('Target')
plt.legend()
plt.show()

#### **Regularization** is a technique to reduce overfitting by adding a penalty to the cost function. In **L2 Regularization** (Ridge Regression), we add the squared magnitude of the weights to the cost function:
#### $$J(\mathbf{w}, b) = \frac{1}{2m} \sum (\hat{y} - y)^2 + \frac{\lambda}{2m} \|\mathbf{w}\|^2$$
#### Where $\lambda$ is the regularization parameter that controls the strength of the penalty. And $\|\mathbf{w}\|^2 = \sum w_i^2$ is the squared magnitude of the weights.
#### This penalizes large values of $w$, forcing the model to prefer smaller weights.
#### The gradient descent must also be updated, but regularization affects only the weight updates, not the bias $b$:
#### $$\frac{\partial J}{\partial w} = \frac{1}{m} \sum (\hat{y} - y)X + \frac{\lambda}{m} \mathbf{w} \qquad \text{and} \qquad \frac{\partial J}{\partial b} = \frac{1}{m} \sum (\hat{y} - y)$$

In [20]:
def regularized_cost(X, Y, w, b, lambda_):
    predictions = mult_lin_reg(X, w, b)
    cost = (1 / (2 * m)) * np.sum((predictions - Y) ** 2)
    reg_term = (lambda_ / (2 * m)) * np.sum(w ** 2) # Regularization term
    return cost + reg_term

In [24]:
def regularized_grad_descent(X, Y, w, b, alpha, lambda_, iter):
    for i in range(iter):
        predictions = mult_lin_reg(X, w, b)
        error = predictions - Y

        # Gradients with regularization
        dw = (1 / m) * np.dot(X.T, error) + (lambda_ / m) * w
        db = (1 / m) * np.sum(error)

        # Update parameters
        w -= alpha * dw
        b -= alpha * db
        
        # Every 100 iterations print the cost
        if i % 100 == 0:
            cost = mult_cost_function(X, Y, w, b)
            print(f"Iteration {i}: Cost {cost:.4f}")
    return w, b

In [22]:
# Initialize parameters
lambda_ = 0.1 # Regularization strength
w_init = np.zeros(n)
b_init = 0

In [ ]:
# Train model
w, b = regularized_grad_descent(X, Y, w_init, b_init, alpha, lambda_, iterations)
print(f"Trained parameters with regularization: w = {w}, b = {b}")

In [ ]:
# Final cost
final_cost = regularized_cost(X, Y, w, b, lambda_)
print(f"Final regularized cost: {final_cost:.4f}")

#### **Feature selection** reduces dimensionality by identifying and retaining the most relevant features. It improves model interpretability and reduces overfitting.
#### Common methods to assess feature relevance:
#### 1. **Correlation**
#### Use Pearson correlation to measure the linear relationship between features and the target, then eliminates features weakly correlated with the target or highly correlated with one another (redundant features).
#### **Steps**
#### 1. Compute correlation coefficients between each feature and the target.
#### 2. Remove features with correlation below a threshold.
#### 3. Optionally, remove one of any pair of features with high inter-correlation.

In [33]:
def select_features_by_correlation(X, Y, target_threshold=0.2, inter_feature_threshold=0.8):
    # Correlation with the target
    target_corr = np.abs(np.corrcoef(X.T, Y))[-1, :-1] # Correlation of features with Y
    selected_indices = np.where(target_corr > target_threshold)[0]

    # Reduce X to selected features
    X_selected = X[:, selected_indices]

    # Inter-feature correlation
    inter_corr = np.corrcoef(X_selected, rowvar=False)
    redudant_features = set()

    for i in range(inter_corr.shape[0]):
        for j in range(i + 1, inter_corr.shape[1]):
            if abs(inter_corr[i, j]) > inter_feature_threshold:
                redudant_features.add(j)
    
    # Remove redundant features
    non_redundant_indices = [i for i in range(X_selected.shape[1]) if i not in redudant_features]
    return X_selected[:, non_redundant_indices], selected_indices[non_redundant_indices]

In [39]:
# Correlation-Based Dataset
X_correlation = np.array([
    [1, 2, 0.5],
    [2, 4, 1],
    [3, 6, 1.5],
    [4, 8, 2]
])  # The third feature is strongly correlated with the second feature.
Y_correlation = np.array([10, 20, 30, 40])  # Target is perfectly correlated with the first feature.


In [ ]:
X_selected, selected_indices = select_features_by_correlation(X_correlation, Y_correlation, target_threshold=0.1, inter_feature_threshold=0.9)
print(f"Correlation-Based - Selected features: {selected_indices}")
print(f"Reduced dataset:\n{X_selected}")

#### **Correlation-Based Feature Selection Visualization**
#### This plot shows:
#### - Correlation of each feature with the target variable.
#### - Inter-feature correlation.

In [ ]:
def plot_correlation_matrix(X, Y, selected_indices):
    plt.figure(figsize=(12, 6))

    # Correlation matrix for all features
    corr_matrix = np.corrcoef(np.hstack([X, Y.reshape(-1, 1)]), rowvar=False)
    target_corr = corr_matrix[:-1, -1]  # Correlation with target

    # Plot target correlations
    plt.bar(range(1, len(target_corr) + 1), np.abs(target_corr), color="skyblue", label="All Features")
    plt.scatter(
        selected_indices + 1,
        np.abs(target_corr[selected_indices]),
        color="red",
        label="Selected Features",
        zorder=3,
    )
    plt.axhline(y=0.2, color="green", linestyle="--", label="Target Threshold")
    plt.title("Correlation-Based Feature Selection")
    plt.xlabel("Feature Index")
    plt.ylabel("Correlation with Target")
    plt.legend()
    plt.show()

plot_correlation_matrix(X_correlation, Y_correlation, selected_indices)

#### 2. **Variance Threshold**
#### Filters features with low variance, as they provide little information about the target.

In [28]:
def select_features_by_variance(X, threshold=0.1):
    variances = np.var(X, axis=0)
    selected_features = np.where(variances > threshold)[0]
    return X[:, selected_features], selected_features

In [42]:
# Variance Threshold Dataset
X_variance = np.array([
    [1, 0.1, 5],
    [1, 0.1, 6],
    [1, 0.1, 7],
    [1, 0.1, 8]
])  # The second feature has very low variance.
Y_variance = np.array([1, 2, 3, 4])  # Target

In [ ]:
X_selected, selected_indices = select_features_by_variance(X_variance, threshold=0.01)
print(f"Variance Threshold - Selected features: {selected_indices}")
print(f"Reduced dataset:\n{X_selected}")

#### **Variance Threshold Visualization**
#### This plot highlights the low-variance feature being removed.

In [ ]:
# Before and after Variance Threshold
def plot_variance_threshold(X_original, X_selected):
    plt.figure(figsize=(10, 6))
    feature_count = X_original.shape[1]
    original_features = range(1, feature_count + 1)

    # Variance of each feature
    variances = np.var(X_original, axis=0)

    # Plot original variances
    plt.bar(original_features, variances, color="skyblue", label="Original Features")
    selected_features = np.var(X_selected, axis=0)
    plt.scatter(
        range(1, len(selected_features) + 1),
        selected_features,
        color="red",
        label="Selected Features",
        zorder=3,
    )
    plt.axhline(y=0.01, color="green", linestyle="--", label="Threshold")
    plt.title("Variance Threshold: Feature Variances")
    plt.xlabel("Feature Index")
    plt.ylabel("Variance")
    plt.legend()
    plt.show()

plot_variance_threshold(X_variance, X_selected)


#### **3. Recursive Feature Elimination (RFE)**
#### Uses a model to iteratively eliminate the least important features based on their contribution to the model’s performance.

In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# Define the model
model = LinearRegression()

# Recursive feature elimination
num_features_to_select = 2 # Number of features to retain
rfe = RFE(estimator=model, n_features_to_select=num_features_to_select)
X_selected = rfe.fit_transform(X, Y)

In [50]:
from sklearn.datasets import make_regression

# RFE Dataset: Generate a regression problem with 5 features
X_rfe, Y_rfe = make_regression(n_samples=100, n_features=5, noise=0.1, random_state=42, coef=False)

In [ ]:
print("\nRFE Dataset (first 5 rows):")
print(X_rfe[:5])

#### **Recursive Feature Elimination (RFE) Visualization**
#### This plot shows the ranking of features after applying RFE.

In [ ]:
def plot_rfe_feature_ranking(rfe, X):
    plt.figure(figsize=(10, 6))
    feature_ranking = rfe.ranking_

    # Plot feature ranking
    plt.bar(range(1, len(feature_ranking) + 1), feature_ranking, color="skyblue", label="Feature Ranking")
    plt.axhline(y=1, color="red", linestyle="--", label="Selected Features")
    plt.title("Recursive Feature Elimination (RFE) Rankings")
    plt.xlabel("Feature Index")
    plt.ylabel("Ranking")
    plt.legend()
    plt.show()

plot_rfe_feature_ranking(rfe, X_rfe)

#### Combining all three methods, one can create a pipeline to combine these feature selection methods for more refined results:
#### 1. **Apply Variance Threshold** to eliminate irrelevant features quickly.
#### 2. **Use Correlation** to filter features based on their relationship with the target and inter-correlation.
#### 3. **Refine with RFE** to rank the remaining features and select the best subset.

### **Comparison of Methods**

| **Method**                  | **Strengths**                                        | **Weaknesses**                                   |
|-----------------------------|----------------------------------------------------|------------------------------------------------|
| **Variance Threshold**      | Fast and simple.                                   | Ignores relationship with the target.          |
| **Correlation**             | Considers relevance to target and redundancy.      | Assumes linear relationships; may ignore non-linear patterns. |
| **Recursive Feature Elimination (RFE)** | Accounts for feature interactions and model importance. | Computationally expensive for large datasets.  |

#### **Use Case Scenarios**
#### - **Small Datasets**: Use all three methods for better feature selection.
#### - **Large Datasets**: Start with **Variance Threshold** and **Correlation** for speed; use **RFE** only if computationally feasible.
#### - **High-Dimensional Datasets**: Combine Variance Threshold and Correlation, and avoid RFE unless highly optimized.

In [58]:
# Extended Combined Dataset
X_combined = np.array([
    [1, 2, 0.1, 0.5, 10],
    [2, 4, 0.1, 1, 20],
    [3, 6, 0.1, 1.5, 30],
    [4, 8, 0.1, 2, 40],
    [5, 10, 0.1, 2.5, 50],
    [6, 12, 0.1, 3, 60],
    [7, 14, 0.1, 3.5, 70],
    [8, 16, 0.1, 4, 80],
    [9, 18, 0.1, 4.5, 90],
    [10, 20, 0.1, 5, 100]
])  # Feature 3 is constant, Feature 4 is strongly correlated with Feature 5.

Y_combined = np.array([15, 30, 45, 60, 75, 90, 105, 120, 135, 150])  # Target is strongly correlated with Feature 5.

In [ ]:
# Step 1: Variance Threshold
X_vt_combined, vt_indices = select_features_by_variance(X_combined, threshold=0.01)

# Step 2: Correlation-Based Selection
X_corr_combined, corr_indices = select_features_by_correlation(
    X_vt_combined, Y_combined, target_threshold=0.1, inter_feature_threshold=0.95
)

# Step 3: Recursive Feature Elimination
if X_corr_combined.shape[1] >= 2:  # Ensure at least 2 features for RFE
    rfe = RFE(estimator=LinearRegression(), n_features_to_select=2)
    X_rfe_combined = rfe.fit_transform(X_corr_combined, Y_combined)
else:
    print("\nSkipping RFE: Not enough features remaining after previous steps.")
    X_rfe_combined = X_corr_combined  # Use the current dataset

if X_corr_combined.shape[1] < 2:
    print("\nOnly one feature remaining after correlation-based selection:")
    print(X_corr_combined)

In [62]:
def plot_selected_features(X, y, title="Final Selected Features"):
    plt.figure(figsize=(10, 6))
    for i in range(X.shape[1]):
        plt.scatter(X[:, i], y, label=f"Feature {i + 1}")

    plt.title(title)
    plt.xlabel("Feature Value")
    plt.ylabel("Target Value")
    plt.legend()
    plt.show()

In [ ]:
if X_rfe_combined.shape[1] > 1:
    plot_selected_features(X_rfe_combined, Y_combined, title="Final Selected Features (Extended Dataset)")
else:
    plt.figure(figsize=(8, 6))
    plt.scatter(X_rfe_combined, Y_combined, color="blue", label="Selected Feature")
    plt.title("Final Selected Feature (Extended Dataset)")
    plt.xlabel("Feature Value")
    plt.ylabel("Target Value")
    plt.legend()
    plt.show()